### Sentiment Analysis

We want to do sentiment analysis by using [VaderSentiment](https://github.com/cjhutto/vaderSentiment) ML framework. The goal of sentiment analysis is to "gauge the attitudes, sentiments, evaluations, and emotions of a speaker/writer based on the computational treatment of subjectivity in a text."

VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.

VADER has a lot of advantages over traditional methods of Sentiment Analysis, including:

 * It works exceedingly well on social media type text, yet readily generalizes to multiple domains
 * It doesn’t require any training data but is constructed from a generalizable, valence-based, human-curated gold standard sentiment lexicon
 * It is fast enough to be used online with streaming data, and
 * It does not severely suffer from a speed-performance tradeoff.
 
 <table>
  <tr><td>
    <img src="https://github.com/dmatrix/olt-mlflow/raw/master/models/images/sentiment_analysis.jpg"
         alt="Sentiment Analysis with Vader" height="400 width="600">
  </td></tr>
</table>

[image source](https://medium.com/analytics-vidhya/sentiment-analysis-with-vader-label-the-unlabeled-data-8dd785225166)

Need to install the package

In [30]:
%pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


### VaderSentiment Python Package

You can read the orignal paper by authors [here](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf).

In [40]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
from ray import serve

Define some input texts we going to analyse 

In [41]:
INPUT_TEXTS = [{'text': "This is a bad ass movie. You got to see it! :-)"},
               {'text': "Ricky Gervais is smart, witty, and creative!!!!!! :D"},
               {'text': "LOL, this guy fell off a chair while sleeping and snoring in a meeting"},
               {'text': "Men shoots himself while trying to steal a dog, OMG"},
               {'text': "Ray and Ray Serve just rocks. Love the way you easily define Ray Actors. Simple APIs, and they work!"},
               {'text': "Yay!! Another good phone interview. I nailed it!!"},
               {'text': "This is INSANE! I can't believe it. How could you do such a horrible thing?"}]

### Start the Ray Serve. It automatically starts Ray processes on the local host

In [42]:
serve.start()

(ServeController pid=33900) 2022-01-27 07:49:21,035	INFO checkpoint_path.py:16 -- Using RayInternalKVStore for controller checkpoint and recovery.
(ServeController pid=33900) 2022-01-27 07:49:21,037	INFO http_state.py:98 -- Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:wWaVYt:SERVE_PROXY_ACTOR-node:127.0.0.1-0' on node 'node:127.0.0.1-0' listening on '127.0.0.1:8000'
2022-01-27 07:49:21,868	INFO api.py:463 -- Started Serve instance in namespace 'serve'.


(HTTPProxyActor pid=35181) INFO:     Started server process [35181]


## Ray Serve Archictecture and components

<img src="https://docs.ray.io/en/latest/_images/architecture.svg" height="50%" width="60%">

There are three kinds of actors that are created to make up a Serve instance:

**Controller**: A global actor unique to each Serve instance that manages the control plane. The Controller is responsible for creating, updating, and destroying other actors. Serve API calls like creating or getting a deployment make remote calls to the Controller.

**Router**: There is one router per node. Each router is a Uvicorn HTTP server that accepts incoming requests, forwards them to replicas, and responds once they are completed.

**Worker Replica**: Worker replicas actually execute the code in response to a request. For example, they may contain an instantiation of an ML model. Each replica processes individual requests from the routers (they may be batched by the replica using `@serve.batch`, see the [batching docs](https://docs.ray.io/en/latest/serve/ml-models.html#serve-batching)).

## Lifetime of a Request¶

When an HTTP request is sent to the router, the follow things happen:

 * The HTTP request is received and parsed.

 * The correct deployment associated with the HTTP url path is looked up. The request is placed on a queue.

 * For each request in a deployment queue, an available replica is looked up and the request is sent to it. If there are no available replicas (there are more than max_concurrent_queries requests outstanding), the request is left in the queue until an outstanding request is finished.

Each replica maintains a queue of requests and executes one at a time, possibly using asyncio to process them concurrently. If the handler (the function for the deployment or __call__) is async, the replica will not wait for the handler to run; otherwise, the replica will block until the handler returns.



### Define a SocialMediaAnalyserModel

This is our main Ray Serve deployment. We can provide route prefix as our path `/sentiments` as part of URL
to send requests to.


In [43]:
@serve.deployment(route_prefix="/sentiments")
class SocialMediaAnalyserModel(object):
    def __init__(self):
      """
      Constructor for our Sentiment Analyser
      """
      # Initialize an instance of vader analyser
      self._analyser = SentimentIntensityAnalyzer()

    async def __call__(self, starlette_request):
        payload = await starlette_request.json()
        text = payload['text']
        print("Worker: received starlette request with sentimet text", text)
        scores = await self._score(text)
        print(f"<{text}> --> {str(scores)}>")
        
    async def _score(self, text):
        """
        Private function to analyse the scores. It invokes model's 
        param: text to analyse
        return: sentiment analyses scores
        """
        scores = self._analyser.polarity_scores(text)
        return scores

## Deploy the model

In [44]:
SocialMediaAnalyserModel.deploy()

2022-01-27 07:49:24,156	INFO api.py:242 -- Updating deployment 'SocialMediaAnalyserModel'. component=serve deployment=SocialMediaAnalyserModel
(ServeController pid=33900) 2022-01-27 07:49:24,256	INFO deployment_state.py:912 -- Adding 1 replicas to deployment 'SocialMediaAnalyserModel'. component=serve deployment=SocialMediaAnalyserModel
2022-01-27 07:49:25,114	INFO api.py:249 -- Deployment 'SocialMediaAnalyserModel' is ready at `http://127.0.0.1:8000/sentiments`. component=serve deployment=SocialMediaAnalyserModel


We can now send HTTP requests to our route `route_prefix=/sentiments` at the default port 8000

In [45]:
import requests  # for making web requests
for sentiment in INPUT_TEXTS:
    response = requests.get(
    "http://localhost:8000/sentiments", json=sentiment)
    print(response.text)








(SocialMediaAnalyserModel pid=35183) Worker: received starlette request with sentimet text This is a bad ass movie. You got to see it! :-)
(SocialMediaAnalyserModel pid=35183) <This is a bad ass movie. You got to see it! :-)> --> {'neg': 0.0, 'neu': 0.542, 'pos': 0.458, 'compound': 0.7644}>
(SocialMediaAnalyserModel pid=35183) Worker: received starlette request with sentimet text Ricky Gervais is smart, witty, and creative!!!!!! :D
(SocialMediaAnalyserModel pid=35183) <Ricky Gervais is smart, witty, and creative!!!!!! :D> --> {'neg': 0.0, 'neu': 0.316, 'pos': 0.684, 'compound': 0.8957}>
(SocialMediaAnalyserModel pid=35183) Worker: received starlette request with sentimet text LOL, this guy fell off a chair while sleeping and snoring in a meeting
(SocialMediaAnalyserModel pid=35183) <LOL, this guy fell off a chair while sleeping and snoring in a meeting> --> {'neg': 0.0, 'neu': 0.786, 'pos': 0.214, 'compound': 0.5473}>
(SocialMediaAnalyserModel pid=35183) Worker: received starlet

## Cleanup

In [46]:
deployments = serve.list_deployments()
print(f'deployments: {deployments}')

deployments: {'SocialMediaAnalyserModel': Deployment(name=SocialMediaAnalyserModel,version=None,route_prefix=/sentiments)}


In [47]:
serve.shutdown()

(ServeController pid=33900) 2022-01-27 07:50:01,685	INFO deployment_state.py:932 -- Removing 1 replicas from deployment 'SocialMediaAnalyserModel'. component=serve deployment=SocialMediaAnalyserModel


## Exercise - Try Adding more examples

Here are some things you can try:

1. Add more neutral, negative and positive texts.
2. Add more replicas and view them in the dashboard